<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/hosting%20server%20in%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install simpletransformers
!pip install torch
!pip install pythainlp
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 17

In [7]:
!rm -rf '/content/ngrok-stable-linux-amd64.tgz'
!rm -rf '/content/ngrok'

!pip install flask --quiet
!pip install flask-ngrok --quiet

# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your AuthToken here and execute this command
!./ngrok authtoken 2Q8ZedCqtQIP2l7e7KVREa2MMvc_2qRULQx4QRLYG97GMTyaZ

--2023-05-22 06:45:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  5.97MB/s    in 2.2s    

2023-05-22 06:46:00 (5.97 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
import logging
import torch
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

device = torch.cuda.is_available()
path = '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr'

model = T5Model("mt5", path, args=model_args, use_cuda=device)

In [15]:
import re
import pycrfsuite
from pythainlp import sent_tokenize

def split_sentence_TH(TH_predict):
  TH_predict = sent_tokenize(TH_predict)
  for i in range(len(TH_predict)):
    # remove space in back
    TH_predict[i] = TH_predict[i].rstrip()

    # debug for engine fail example ['750ml', '1500ml']
    # move to append previous sentences
    # and replace ''
    if re.search('[0-9]', TH_predict[i][0]):
      j = i
      while True:
        if TH_predict[j-1] != '':
          TH_predict[j-1] += ' ' + TH_predict[i]
          break
      j -= 1
      TH_predict[i] = ''

  # remove '' in list
  while '' in TH_predict:
    TH_predict.remove('')
  print('[split sentence completed]')
  print(TH_predict)
  return TH_predict

def split_sentence_EN(EN_predict):
  EN_predict = EN_predict.split('. ')
  for i in range(len(EN_predict)):
    if EN_predict[i][-1] != '.':
      EN_predict[i] += '.'
  return EN_predict
def predict(text):
  predictions = model.predict(text)
  print('[split sentence completed]')
  print(predictions)
  return predictions

In [17]:
# import Flask from flask module
from flask import Flask, request, jsonify

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__) #app name
run_with_ngrok(app)
  
@app.route("/", methods=["GET", "POST"])
def index():
  if request.method == "POST":
    json_data = request.get_json()
    if not json_data:
      return jsonify({"error": "no data"})
    try:
      if json_data["task"] == "th-en":
        prediction = split_sentence_TH(json_data["text"])
      elif json_data["task"] == "en-th":
        prediction = split_sentence_TH(json_data["text"])
      prediction = predict(prediction)
      data_predicted = {"prediction": prediction}
      return jsonify(data_predicted)
    except Exception as e:
      return jsonify({"error": str(e)})
  return "OK"
  
if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f899-35-221-221-44.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
[split sentence completed]
['This engineering project develops a system for detecting. and classifying plant disease from photographs of melon leaves using deep learning.', 'The system uses NVIDIA Jetson Nano to process images from cameras in the greenhouse. To develop a deep learning model, the images of the melon leaves from the greenhouses were collected. and the melon leaf disease were labelled. Then use the dataset to train the object detector model. which will be able to detect disease in melon leaves. and when the system detects a disease, the caretaker will be notified of the issue through the LINE application. In the process of selecting a deep learning model to be deployed on the Jetson Nano.']


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3732: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [22/May/2023 07:05:59] "POST / HTTP/1.1" 200 -


[split sentence completed]
['โครงการวิศวกรรมนี้พัฒนาระบบตรวจจับและจําแนกโรคพืชจากภาพถ่ายใบมะกรูดโดยใช้การเรียนรู้ลึก', 'ระบบใช้ NVIDIAjetsonNANO เพื่อประมวลผลภาพจากกล้องในโรงเรือนเพื่อพัฒนาแบบจําลองการเรียนรู้ลึก นําภาพใบมะกรูดจากโรงเรือนมาเก็บรวบรวมและระบุโรคมะม่วงจากโรงเรือน เพื่อพัฒนาแบบจําลองการเรียนรู้ลึก ที่สามารถตรวจจับโรคในใบมะกรูด และเมื่อระบบตรวจจับโรคในใบมะกรูด เจ้าหน้าที่จะแจ้งเตือนปัญหาผ่านแอพพลิเคชั่น LINE ในขั้นตอนของการคัดเลือกแบบจําลองการเรียนรู้ลึก']


INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:11:23] "POST / HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:12:53] "POST / HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:13:03] "POST / HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:16:14] "POST / HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:46:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 08:46:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 09:45:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 09:45:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 09:46:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2023 09:46:05] "GET /favicon.ico HTTP/1.1" 404 -
